### 12 Jan 2023

### This code is almost entirely written by Dr. Tasha Snow. Minor edits from Elena Savidge.

# Code for seal data processing. 
Once these cells are run for each year, the output pickle files (e.g., named 'seal19_file') can be used for making all figures.

In [2]:
from glob import glob
import pandas as pd
import re
import os
import gsw
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import pandas as pd
import netCDF4
#import pickle

In [3]:
pwd

'/Users/elenasavidge/Documents/Documents - Elena’s MacBook Pro/Sealtag/savidge23_manuscriptcode'

### Define functions

In [4]:
def read_nc(fname):
    '''Read in NetCDF files. Add station and datetime columns and calculate depth.'''
    
    # Read in netcdf variables 
    seal = Dataset(fname, mode='r')
    name = os.path.basename(fname).split('.')[0]
    
    lat = seal.variables['LATITUDE'][:]
    lon = seal.variables['LONGITUDE'][:]
    temp = seal.variables['TEMP_ADJUSTED'][:]
    pres = seal.variables['PRES_ADJUSTED'][:]
    try:
        psal = seal.variables['PSAL_ADJUSTED'][:]
    except:
        psal = np.zeros((temp.shape[0],temp.shape[1]))
    jday = seal.variables['JULD'][:]
    
    timetrans = pd.DataFrame((pd.to_datetime('1950-01-01') + pd.to_timedelta(jday,unit='d')),columns=['Datetime'])
    latlon= pd.concat([pd.DataFrame(lat,columns=['LATITUDE']),pd.DataFrame(lon,columns=['LONGITUDE'])],axis=1)

    # Flatten 2d dataframes so each station's depth measurement has its own row, and concatenate 
    temp_frame = pd.concat([timetrans,latlon,pd.DataFrame(temp)],axis=1)
    temp_frame = pd.melt(temp_frame,id_vars=['Datetime','LATITUDE','LONGITUDE'],value_name='Temperature [C]')
    temp_frame = temp_frame.set_index('Datetime')

    psal_frame = pd.concat([timetrans,pd.DataFrame(psal)],axis=1)
    psal_frame = pd.melt(psal_frame,id_vars=['Datetime'],value_name='Salinity [PSU]')
    psal_frame = psal_frame.set_index('Datetime')

    pres_frame = pd.concat([timetrans,pd.DataFrame(pres)],axis=1)
    pres_frame = pd.melt(pres_frame,id_vars=['Datetime'],value_name='Pressure [dbar]')
    pres_frame = pres_frame.set_index('Datetime')

    cast = pd.concat([temp_frame,psal_frame,pres_frame],axis=1)
    cast['Station'] = name

    cast = cast.reset_index()
    cast = cast.set_index('Pressure [dbar]')
    cast = cast.drop(columns = ['variable'],axis=1)
    
    # Remove bad pressure columns
    cast = cast[cast.index.notnull()]
    
    # Calculate depth so can reindex to that
    cast['z'] = -gsw.z_from_p(cast.index, cast['LATITUDE'])
    
    return cast

In [5]:
def proc_seal(cast,dive_type='seal'):
    '''Process CTD DataFrame. Meant for seal data. Needs to be one cast at a time.
    
    cast = DataFrame
    colindex = index interpolated across, by depth 'z' or 'Pressure [dbar]'
    
    cast columns of Datetime and Station will be removed and readded so as to not mess up interp 
    '''
#     cast = cast.reset_index()
#     cast = cast.set_index(cast.index)

    dt = cast.Datetime.min()
    name = cast.Station.min()
    try:
        cast = cast.drop(columns = ['Datetime','Station'],axis=1)
    except:
        cast = cast.drop(columns = ['Datetime'],axis=1)
    
    if dive_type=='seal':
        # Interpolate - cubic spline
        cast = seal_interp(cast,1,dive_type)

    # 08-Derive.
    cast = derive_csv(cast) 
    try:
        cast['Datetime'] = dt
        cast['Station'] = name
    except:
        cast['Datetime'] = dt
    
    return cast

In [6]:
def seal_interp(df,dz=1,dive_type='seal'):
    '''Interpolates all columns in a DataFrame based on index of either depth or pressure at intervals of dz (m). 
    Meant for seal CTD data. Cannot include strings/non-interprolationable data'''
    if dive_type=='seal':
        # New index to interpolate to goes down to max depth
        x2 = np.arange(0,df.index.max()+1,dz)
    if dive_type=='glider':
        x2 = np.arange(np.round(df.index.min()),df.index.max()+1,dz)
    col = df.columns
    # Interpolate using gsw's pchip interpolation, will get a wrong data error if index is not increasing
    # Interpolation from surface to max depth +1. Pchip errors if too few point, then this reverts to linear interp
    try: 
        y2 = [gsw.pchip_interp(df.index, df.iloc[:, i],x2) for i in range(len(col))]
    except:
        print ('Too few points - reverted to linear interpolation')
        y2 = [np.interp(x2,df.index, df.iloc[:, i]) for i in range(len(df.columns))]  
    stack = np.column_stack(y2)
    return pd.DataFrame(stack, index=x2,columns=col)

In [7]:
def derive_csv(self):
    """Compute SA, PT, and sigma0 from a csv pre-processed cast."""
    cast = self.copy()  # FIXME: Use MetaDataFrame to propagate lon, lat.
    p = cast.index.values.astype(float)
    cast['SA'] = gsw.SA_from_SP(cast['Salinity [PSU]'].values, p, cast['LONGITUDE'].values,  cast['LATITUDE'].values)
    cast['CT'] = gsw.CT_from_t(cast['SA'].values, cast['Temperature [C]'].values, p)
    cast['PT'] = gsw.pt0_from_t(cast['SA'].values, cast['Temperature [C]'].values, p)
    cast['z'] = -gsw.z_from_p(p, cast['LATITUDE'])
    cast['PD'] = gsw.sigma0(cast['SA'].values, cast['CT'].values)
    cast['N2'] = np.hstack([np.nan, gsw.Nsquared(cast['SA'].values, cast['CT'].values, p)[0]])
    return cast

### Run functions

#### 2014 seal data

'Too few points - reverted to linear interpolation' -- some dives in 2014 do not have enough points for interpolation. That is a normal message.

In [8]:
# 2014 data

# Point to file names
fnames = glob('2014sealdata/ct*.nc')

try:
    del seal14
except:
    print ('no old variables to delete')

# Read and process all nc files in a directory, accounts for multiple dives in one file via date, dif locations per 
# date, and god forbid multiple dives on same day and location 
for fname in fnames:
    sealcat = []
    seal_data = read_nc(fname)
    name = os.path.basename(fname).split('.')[0]
    print (name)
    for dt in seal_data.Datetime.unique():
        dive_d = seal_data[seal_data['Datetime']==dt]
        for lat in dive_d.LATITUDE.unique():
            dive_l = dive_d[dive_d['LATITUDE']==lat]
            dup = 0
            for i in dive_l.index.duplicated(keep='first')[1:]:
                if i == False: 
                    break
                dup+=1
            for i in range(dup+1):
                dive = dive_l.iloc[i::(dup+1), :] # starting on i, at intervals of dup+1
                if dive.shape[0]>0:
                    dive = proc_seal(dive) #with interpolation
#                     dive = proc_seal(dive,"no int") #without interpolation
                    try: 
                        sealcat = pd.concat([sealcat,dive])
                    except:
                        sealcat = dive
                else:
                    print ((str(dt) + ' has no data'))
    # Makes faster
    try:
        seal14 = pd.concat([seal14,sealcat])
    except:
        seal14 = sealcat
                    
# Pickle data
seal14.to_pickle('seal14_file')
# seal14.to_pickle('seal49NI_file')

seal14.head(2)

no old variables to delete
ct104-EM889-13_prof 1
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
Too few points - reverted to linear interpolation
c

,LATITUDE,LONGITUDE,Temperature [C],Salinity [PSU],z,SA,CT,PT,PD,N2,Datetime,Station
0.0,-73.985466,-103.313785,-0.71486,33.174992,0.000000,33.334178,-0.708218,-0.714860,26.668610,NaN,2014-02-12 05:30:59.999990400,ct104-EM889-13_prof 1
1.0,-73.985466,-103.313785,-0.71486,33.174992,0.989659,33.334161,-0.708243,-0.714885,26.668598,-1.154761e-07,2014-02-12 05:30:59.999990400,ct104-EM889-13_prof 1


#### 2019 seal data

In [31]:
# 2019 data

# Point to file names
fnames = glob('2019sealdata/ct147/ct147*')

try:
    del seal19
except:
    print ('no old variables to delete')
    
# Read and process all nc files in a directory, accounts for multiple dives in one file via date, dif locations per 
# date, and god forbid multiple dives on same day and location 
for fname in fnames:
    sealcat = []
    seal_data = read_nc(fname)
    name = os.path.basename(fname).split('.')[0]
    print (name)
    for dt in seal_data.Datetime.unique():
        dive_d = seal_data[seal_data['Datetime']==dt]
        for lat in dive_d.LATITUDE.unique():
            dive_l = dive_d[dive_d['LATITUDE']==lat]
            dup = 0
            for i in dive_l.index.duplicated(keep='first')[1:]:
                if i == False: 
                    break
                dup+=1
            for i in range(dup+1):
                dive = dive_l.iloc[i::(dup+1), :] # starting on i, at intervals of dup+1
                if dive.shape[0]>0:
                    dive = proc_seal(dive) #with interpolation
#                     dive = proc_seal(dive,"no int") #without interpolation
                    try: 
                        sealcat = pd.concat([sealcat,dive])
                    except:
                        sealcat = dive
                else:
                    print ((str(dt) + ' has no data'))
    # Makes faster
    try:
        seal19 = pd.concat([seal19,sealcat])
    except:
        seal19 = sealcat
                    
# Pickle data
seal19.to_pickle('seal19_file')

seal19.head(2)

no old variables to delete
ct147-867-18_prof
ct147-898-18_prof
ct147-871-18_prof
ct147-897-18_prof
ct147-868-18_prof
2019-03-12T05:30:00.000028800 has no data
ct147-894-18_prof
2019-03-05T03:29:59.999971200 has no data
ct147-872-18_prof
ct147-891-18_prof
ct147-863-18_prof
ct147-864-18_prof
2019-03-04T23:10:00.000019200 has no data
2019-03-05T04:09:59.999990400 has no data
ct147-869-18_prof
ct147-870-18_prof


,LATITUDE,LONGITUDE,Temperature [C],Salinity [PSU],z,SA,CT,PT,PD,N2,Datetime,Station
0.0,-73.664004,-103.15514,-0.323368,33.512524,0.000000,33.673292,-0.317368,-0.323368,26.924812,NaN,2019-02-12 22:59:59.999971200,ct147-867-18_prof
1.0,-73.664004,-103.15514,-0.323368,33.512524,0.989675,33.673276,-0.317398,-0.323397,26.924801,-1.101809e-07,2019-02-12 22:59:59.999971200,ct147-867-18_prof


#### 2020 seal data

In [12]:
# 2020 data

# Point to file names
fnames = glob('2020sealdata/ct153/ct153*')

try:
    del seal20
except:
    print ('no old variables to delete')
    
# Read and process all nc files in a directory, accounts for multiple dives in one file via date, dif locations per 
# date, and god forbid multiple dives on same day and location 
for fname in fnames[1:]:
    sealcat = []
    seal_data = read_nc(fname)
    name = os.path.basename(fname).split('.')[0]
    print (name)
    for dt in seal_data.Datetime.unique():
        dive_d = seal_data[seal_data['Datetime']==dt]
        for lat in dive_d.LATITUDE.unique():
            dive_l = dive_d[dive_d['LATITUDE']==lat]
            dup = 0
            for i in dive_l.index.duplicated(keep='first')[1:]:
                if i == False: 
                    break
                dup+=1
            for i in range(dup+1):
                dive = dive_l.iloc[i::(dup+1), :] # starting on i, at intervals of dup+1
                if dive.shape[0]>0:
                    dive = proc_seal(dive) #with interpolation
#                     dive = proc_seal(dive,"no int") #without interpolation
                    try: 
                        sealcat = pd.concat([sealcat,dive])
                    except:
                        sealcat = dive
                else:
                    print ((str(dt) + ' has no data'))
    # Makes faster
    try:
        seal20 = pd.concat([seal20,sealcat])
    except:
        seal20 = sealcat
                    
# Pickle data
seal20.to_pickle('seal20_file')
# seal20.to_pickle('seal20NI_file')

seal20.head(2)

ct153-Atreyu-19_prof
ct153-Alice-19_prof
2020-02-27T05:30:00.000028800 has no data
2020-02-27T07:50:00.000009600 has no data
2020-02-27T14:19:59.999980800 has no data
2020-02-27T19:59:59.999971200 has no data
2020-02-27T20:30:00.000028800 has no data
ct153-Lucas-18_prof
ct153-Akira-19_prof_corrected
ct153-Alana-19_prof_corrected
2020-02-27T03:20:00.000009599 has no data
2020-02-27T03:40:00.000019200 has no data
ct153-Pendragon-19_prof
ct153-Jiji-19_prof


,LATITUDE,LONGITUDE,Temperature [C],Salinity [PSU],z,SA,CT,PT,PD,N2,Datetime,Station
0.0,-73.852594,-102.965683,0.347,33.609119,0.000000,33.770362,0.353142,0.347000,26.969078,NaN,2020-02-06 17:19:59.999980800,ct153-Atreyu-19_prof
1.0,-73.852594,-102.965683,0.347,33.609119,0.989666,33.770344,0.353106,0.346964,26.969066,-1.195851e-07,2020-02-06 17:19:59.999980800,ct153-Atreyu-19_prof
